In [ ]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as st
import csv
import sys
import os
from tqdm import tqdm
import warnings
import itertools

datadir = 'data/'
figdir = 'figures/'

#Algorithm 2 (with discrete responses)
def collider_synth(X,max_feats=30,K=30,k=5,num_Y=15,prob_collide = 0.5, sd_X =1, mu_X = 1, sd_Y=0.5,mu_Y=5, intercept = -6):    
    nonzeros = np.where(X.sum(0) > 400)[0]
    X = X[:, nonzeros]
    np.random.shuffle(X)
    top_feats = np.argsort(X.sum(0))[-max_feats:]  
    sig_feats = top_feats[np.random.choice(range(max_feats), K, replace=False)]

    # k = 10  # number of significant features for each Y_j
    x_feats = []
    y_feats = []
    y_coefs = []
    Y = np.zeros((X.shape[0],num_Y))
    for j in range(num_Y):
        x_feats.append(sig_feats[np.random.choice(range(K), k, replace=False)])
        y_depend = np.random.binomial(1,prob_collide,j)
        y_feats.append(np.nonzero(y_depend))
        y_coefs.append(np.random.randn(len(y_feats[j][0]))*sd_Y + mu_Y)


    x_coefs = (np.random.randn(k*num_Y)*sd_X + mu_X).reshape(num_Y,k)


    for j in range(num_Y):
        logodds = intercept + np.matmul(X[:,x_feats[j]] , x_coefs[j].T)
        if(y_coefs[j].shape[0] > 0 ):
            logodds = logodds + np.matmul(Y[:,y_feats[j][0]] , y_coefs[j].T)
        like = 1 / (1 + np.exp(-logodds))
        Y[:,j] = np.random.binomial(1,like)
    return X, Y, x_feats, y_feats

#Algorithm 2 (with continuous responses)
def collider_cont(X,max_feats=30,K=30,k=5,num_Y=15,prob_collide = 0.5, sd_X =1, mu_X = 1, sd_Y=0.5,mu_Y=1, intercept = 0,sd=1):    
    nonzeros = np.where(X.sum(0) > 400)[0]
    X = X[:, nonzeros]
    np.random.shuffle(X)
    top_feats = np.argsort(X.sum(0))[-max_feats:]  
    sig_feats = top_feats[np.random.choice(range(max_feats), K, replace=False)]

    # k = 10  # number of significant features for each Y_j
    x_feats = []
    y_feats = []
    y_coefs = []
    Y = np.zeros((X.shape[0],num_Y))
    Y_response = np.zeros((X.shape[0],num_Y))
    for j in range(num_Y):
        x_feats.append(sig_feats[np.random.choice(range(K), k, replace=False)])
        y_depend = np.random.binomial(1,prob_collide,j)
        y_feats.append(np.nonzero(y_depend))
        y_coefs.append(np.random.randn(len(y_feats[j][0]))*sd_Y + mu_Y)


    x_coefs = (np.random.randn(k*num_Y)*sd_X + mu_X).reshape(num_Y,k)


    for j in range(num_Y):
        response = intercept + np.matmul(X[:,x_feats[j]] , x_coefs[j].T)
        if(y_coefs[j].shape[0] > 0 ):
            response = response + np.matmul(Y[:,y_feats[j][0]] , y_coefs[j].T)
        Y_response[:,j] = response
        Y[:,j] = np.random.normal(response,sd)
    return X, Y,Y_response, x_feats, y_feats

#Algorithm 3 (real-world confoduning)
def semi_synth(X, Y, max_feats, K=30, k=10,truncate=True):
    if(truncate == True):
        nonzeros = np.where(X.sum(0) > 400)[0]
        X = X[:, nonzeros]
    np.random.shuffle(X)
    np.random.shuffle(Y)
    top_feats = np.argsort(X.sum(0))[-max_feats:]  
    sig_feats = top_feats[np.random.choice(range(max_feats), K, replace=False)]

    # k = 10  # number of significant features for each Y_j
    y_feats = []
    for j in range(Y.shape[1]):
        y_feats.append(sig_feats[np.random.choice(range(K), k, replace=False)])
    print(y_feats)
    y_coefs = abs(np.random.randn(X.shape[1]*Y.shape[1]).reshape(Y.shape[1], X.shape[1]))+2
    coef_mask = np.zeros(y_coefs.shape)
    for j in range(len(y_feats)):
        coef_mask[j, y_feats[j]] = 1

    y_coefs = y_coefs * coef_mask  # zero out null features
    like = 1 / (1 + np.exp(-np.matmul(X, y_coefs.T)))
    Y_tilde = np.zeros_like(Y)

    for i in range(X.shape[0]):
        like_i = like[i:i+1]
        prob_y = like_i * Y + (1-like_i) * (1 - Y)
        prob_y = prob_y.sum(1)
        prob_y = prob_y / prob_y.sum()
        Y_tilde[i, :] = Y[np.random.choice(range(Y.shape[0]), p=prob_y)]
    return X, Y_tilde, y_feats

# Merge genes and metastases together
Two versions of the dataset are maintained. 
- The "actual" dataset merges together the genes and metastases together without including primary site information. As these sites are useful from a predictive modelling perspective, these views are mostly only useful as a base for constructing semi-synthetic data. 
- The "actualsmall" dataset includes primary site information and is mostly useful when extracting causal $p$-values on the actual dataset that have biologically interesting implications. 

In [ ]:
#full dataset
genes = pd.read_csv(datadir + 'missing_gene_alteration_matrix.csv',index_col=0)
metastases = pd.read_csv(datadir + 'metastasis_bysamples.csv',index_col=0)
ds = genes.merge(metastases,on="uid",how="left")
ds.to_pickle(datadir + 'merged_dataset.pkl')
nomissing = ds.dropna(axis='columns')
ds = nomissing.to_numpy()
X = nomissing.iloc[:,:331].to_numpy()
Y = nomissing.iloc[:,332:].to_numpy()
np.save(datadir + "actual_x",X)
np.save(datadir + "actual_y",Y)

#reduced dataset
primary_site = pd.read_csv(datadir + 'primary_sites.csv')
counts = primary_site.reset_index().groupby(["PRIMARY_SITE"]).count()['index'].reset_index()
counts = counts[counts['index'] > 200]
primary_site = primary_site.merge(counts, how='left',on="PRIMARY_SITE")
primary_site['include'] = ~primary_site['index'].isnull()
primary_site.index = genes.index
primary_site = primary_site[['PRIMARY_SITE','include']]
ds = genes.merge(primary_site,on="uid",how="left")
ds =ds.merge(metastases,on="uid",how="left")
ds = ds.dropna(axis='columns')
select = np.where(ds.iloc[:,333])[0]
X = ds.iloc[:,:332]
X = X.iloc[:,np.where(X.sum(0) > 400)[0]]
X = X.iloc[select,:]
prim = pd.get_dummies(ds.iloc[select,332])
Y = ds.iloc[select,334:]
np.save(datadir + "actualsmall_x",np.concatenate((X.to_numpy(),prim.iloc[:,1:].to_numpy()),axis=1))
np.save(datadir + "actualsmall_y",Y)
ds.iloc[select,332].to_pickle(datadir + "actualsmall_primary.pkl")  

# Create semi-synthetic datasets
The naming convention is as follows: 
* semisynth-numX-numY refers to discrete datasets using real-world confondunding where num_X refers to the numbers of nodes used within the X dataset and num_Y refers to the number of nodes Y dataset. 
* collidesynth-numX-numY refers to discrete datasets using synthetic confounding where num_X refers to the numbers of nodes used within the X dataset and num_Y refers to the number of nodes Y dataset. Confoundedness parameter set at 0.5. 
* synth_collide_large_p refers to discrete datasets using synthetic confounding where $p$ refers to the confoundedness parameter. num_X set to 27 and num_y set to 15. 
* synth_collide_cont_p refers to continuous datasets using synthetic confounding where $p$ refers to the confoundedness parameter. num_X set to 27 and num_y set to 15. 

. 

In [ ]:
X = ds.iloc[:,:332]
X = X.iloc[:,np.where(X.sum(0) > 750)[0]].to_numpy()
Y = ds.iloc[:,334:].to_numpy()

synth_x, synth_y, synth_true = semi_synth(X, Y, 20, K=20, k=5)
np.save(datadir + "semisynth-20-20_x",synth_x)
np.save(datadir + "semisynth-20-20_y",synth_y)
np.save(datadir + "semisynth-20-20_ytrue",synth_true)

synth_x, synth_y, synth_true = semi_synth(X[:,:15], Y[:,:15], 15, K=15, k=4)
np.save(datadir + "semisynth-15-15_x",synth_x)
np.save(datadir + "semisynth-15-15_y",synth_y)
np.save(datadir + "semisynth-15-15_ytrue",synth_true)

synth_x, synth_y, synth_true = semi_synth(X[:,:10], Y[:,:10], 10, K=10, k=3)
np.save(datadir + "semisynth-10-10_x",synth_x)
np.save(datadir + "semisynth-10-10_y",synth_y)
np.save(datadir + "semisynth-10-10_ytrue",synth_true)

synth_x, synth_y, synth_true = semi_synth(X[:,:5], Y[:,:5], 5, K=5, k=2)
np.save(datadir + "semisynth-5-5_x",synth_x)
np.save(datadir + "semisynth-5-5_y",synth_y)
np.save(datadir + "semisynth-5-5_ytrue",synth_true)

collide_synth_x, collide_synth_y, collide_synth_xtrue, collide_synth_ytrue = collider_synth(X[:,:5],prob_collide = 0.3,max_feats=5,K=5,k=2,num_Y=5)
np.save(datadir + "collidesynth-5-5_x",collide_synth_x)
np.save(datadir + "collidesynth-5-5_y",collide_synth_y)
np.save(datadir + "collidesynth-5-5_xtrue",collide_synth_xtrue)
np.save(datadir + "collidesynth-5-5_ytrue",collide_synth_ytrue)

collide_synth_x, collide_synth_y, collide_synth_xtrue, collide_synth_ytrue = collider_synth(X[:,:10],prob_collide = 0.3,max_feats=10,K=10,k=3,num_Y=10)
np.save(datadir + "collidesynth-10-10_x",collide_synth_x)
np.save(datadir + "collidesynth-10-10_y",collide_synth_y)
np.save(datadir + "collidesynth-10-10_xtrue",collide_synth_xtrue)
np.save(datadir + "collidesynth-10-10_ytrue",collide_synth_ytrue)


collide_synth_x, collide_synth_y, collide_synth_xtrue, collide_synth_ytrue = collider_synth(X[:,:15],prob_collide = 0.3,max_feats=15,K=15,k=4,num_Y=15)
np.save(datadir + "collidesynth-15-15_x",collide_synth_x)
np.save(datadir + "collidesynth-15-15_y",collide_synth_y)
np.save(datadir + "collidesynth-15-15_xtrue",collide_synth_xtrue)
np.save(datadir + "collidesynth-15-15_ytrue",collide_synth_ytrue)

collide_synth_x, collide_synth_y, collide_synth_xtrue, collide_synth_ytrue = collider_synth(X[:,:20],prob_collide = 0.3,max_feats=20,K=20,k=5,num_Y=20)
np.save(datadir + "collidesynth-20-20_x",collide_synth_x)
np.save(datadir + "collidesynth-20-20_y",collide_synth_y)
np.save(datadir + "collidesynth-20-20_xtrue",collide_synth_xtrue)
np.save(datadir + "collidesynth-20-20_ytrue",collide_synth_ytrue)


for i in range(0,10,1):
    collide_synth_x, collide_synth_y, collide_synth_xtrue, collide_synth_ytrue = collider_synth(X,num_Y=15,prob_collide = i/10)
    np.save(datadir + "synth_collide_large_" +str(i/10) + "_x",collide_synth_x)
    np.save(datadir + "synth_collide_large_" +str(i/10)  + "_y",collide_synth_y)
    np.save(datadir + "synth_collide_large_" +str(i/10) + "_xtrue",collide_synth_xtrue)
    np.save(datadir + "synth_collide_large_" +str(i/10) + "_ytrue",collide_synth_ytrue)
    np.savetxt(datadir + "synth_collide_large_" +str(i/10) + "_x.csv",collide_synth_x, delimiter=",")
    np.savetxt(datadir + "synth_collide_large_" +str(i/10) + "_y.csv",collide_synth_y, delimiter=",")
    np.savetxt(datadir + "synth_collide_large_" +str(i/10) + "_xtrue.csv",collide_synth_xtrue, delimiter=",")
    


for i in range(0,10,1):
    collide_synth_x, collide_synth_y, collide_synth_response, collide_synth_xtrue, collide_synth_ytrue = collider_cont(X,num_Y=15,prob_collide = i/10)
    np.save(datadir + "synth_collide_cont_" +str(i/10) + "_x",collide_synth_x)
    np.save(datadir + "synth_collide_cont_" +str(i/10)  + "_y",collide_synth_y)
    np.save(datadir + "synth_collide_cont_" +str(i/10) + "_xtrue",collide_synth_xtrue)
    np.save(datadir + "synth_collide_cont_" +str(i/10) + "_ytrue",collide_synth_ytrue)
    np.savetxt(datadir + "synth_collide_cont_" +str(i/10) + "_x.csv",collide_synth_x, delimiter=",")
    np.savetxt(datadir + "synth_collide_cont_" +str(i/10) + "_y.csv",collide_synth_y, delimiter=",")
    np.savetxt(datadir + "synth_collide_cont_" +str(i/10) + "_xtrue.csv",collide_synth_xtrue, delimiter=",")